In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from transformers import AutoTokenizer
from datasets import Dataset, load_dataset

from pathlib import Path

In [ ]:
data_dir = Path("../data/interim/")
ckpt = "distilbert-base-uncased"

## Tokenize text

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(ckpt, use_fast=True)

In [ ]:
df = pd.read_csv(data_dir/"wndp-api-ohe.csv")
df.columns = df.columns.str.lower().str.replace(" ", "_")
df.dropna(inplace=True)
df.head()

In [ ]:
df.shape

In [ ]:
sample = df.text[:5]
sample[3]

In [ ]:
tokenizer(sample[3])

In [ ]:
tokens = tokenizer.tokenize(sample[3])
tokens

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

In [ ]:
tokenizer.decode(ids)

In [ ]:
tokenizer.decode(tokenizer(sample[3])["input_ids"])

## Build the dataset

In [ ]:
# Pick only the columns we need for the model
df = df[list(set(df.columns).difference({"patient_id", "terms"}))]
df.head()

In [ ]:
# Split dataset into train, val & test set
_ds = (Dataset
          .from_pandas(df)
          .train_test_split(test_size=0.15, 
                            shuffle=True)
     )
ds = _ds["train"].train_test_split(test_size=0.2)
ds["val"] = ds.pop("test")
ds["test"] = _ds["test"]

In [ ]:
ds = ds.remove_columns(column_names=["__index_level_0__"])

In [ ]:
labels = sorted(ds["train"].column_names)
labels.remove("text")
labels

In [ ]:
# Add labels to the dataset as float (pytorch excepts float tensors)
ds = ds.map(lambda row: {"labels": [float(row[l]) for l in labels]})

In [ ]:
ds["train"][0]

Let us look at how long is each of the text description

In [ ]:
text_stats = np.array([len(row.split()) if row else 0 for row in ds["train"]["text"]])

In [ ]:
sns.histplot(text_stats, bins=20, kde=True)

In [ ]:
(text_stats < 3).sum()/len(text_stats)

In [ ]:
(text_stats > 128).sum()/len(text_stats)

1% of the data have words more than 128 words

In [ ]:
for row in ds["train"]["text"]:
    if len(row.split()) < 3:
        print(row)

In [ ]:
for row in ds["train"]["text"]:
    if len(row.split()) > 200:
        print(len(row.split()), row)
        print("-" * 20)

In [ ]:
%%time
def tok_fn(row):
    return tokenizer(row["text"], 
                     truncation=True, 
                     padding="max_length", 
                     max_length=128)

tok_ds = ds.map(tok_fn, batched=True, num_proc=16, remove_columns=labels + ["text"])

In [ ]:
tok_ds

In [ ]:
tok_ds["train"][0]

In [ ]:
# Save the processed data in a parquet file
for split,split_ds in tok_ds.items():
    split_ds.to_parquet(f"../data/processed/wndp-api-data-{split}.parquet")

In [ ]:
!ls ../data/processed/

In [ ]:
%%time
data_files = {
    "train": "../data/processed/wndp-api-data-train.parquet",
    "val": "../data/processed/wndp-api-data-val.parquet",
    "test": "../data/processed/wndp-api-data-test.parquet",
}

ds = load_dataset("parquet", data_files=data_files)